# Simply Rational - ASSIGNMENT 1: Comparative Model Analysis
### Notebook created by: Jiacheng Yao, 09/02/2021



-------------
ASSIGNMENT 1: Comparative Model Analysis

Introduction
Attached to this email you will find two scientific papers describing two types of “simple algorithms” – Fast-And-Frugal Trees (Philipps et al, 2017) and Select-Regress-and-Round (Jung et al, 2020). Please include at least one of these models in your comparative model analysis.


The scenario
Many people struggle to get loans due to insufficient or non-existent credit histories. Unfortunately, this population is often taken advantage of by untrustworthy lenders and credit sharks. An organization strives to broaden financial inclusion for the unbanked population by providing a positive and safe borrowing experience. To make sure this underserved population has a positive loan experience, the organization makes use of a variety of alternative data—including telco and transactional information—to predict their clients' repayment abilities.

In addition, the organization currently operates in a country in which great economic and societal changes are taking place. Factors predictive of successful loan repayment or of loan default today may no longer be predictive a year from now. In order to be able to continue smooth operations during this time of upheaval, the organization wishes to also assess the effectiveness of simple and transparent algorithms. This would allow the employees of the organization to effectively evaluate the predictions made by the new system and integrate their knowledge of these societal and economic changes into the decision-making process.




Assessment of the task
Using ten-fold cross validation, determine the predictive accuracy (balanced accuracy BACC) with regards to TARGET in the main data set “application.csv”. Please use at least one of the simple models and compare it to the other models of your choice.

Your performance will be evaluated along two metrics:

1. Predictive accuracy of your best model.
2. A discussion regarding simplicity and interpretability of the simple models vs black box models and under which circumstances you would recommend the use of the simple models.

Please prepare the code as well as slides for a presentation that showcase your approach, the results and recommendations.

Data Description

1.  application.csv

    This is the main table.
    Static data for all applications. One row represents one loan in our data sample.

2.  bureau.csv

    All previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).
    For every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau before the application date.
    There are more IDs in this file than in the “application.csv”-file. Please ignore those IDs that are not included in “application.csv”.

3.  columns_description.csv

    This file contains descriptions for the columns in the various data files.
    
## 1. Explorative Analysis

In [1]:
import logging

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("Start 1. Explorative Analysis")

INFO:root:Start 1. Explorative Analysis


### 1. Read the input data

In [2]:
df_app = pd.read_csv("application.csv")

df_bureau = pd.read_csv("bureau.csv", sep = ",")

df_col_des = pd.read_csv("columns_description.csv", encoding="ISO-8859-1")

### 2. Take a first look at the data - Application:

In [3]:
df_app.head(10)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,random_number
0,"100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
1,100003,0.0,Cash loans,F,N,N,0.0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,100004,0.0,Revolving loans,M,Y,Y,0.0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,100006,0.0,Cash loans,F,N,Y,0.0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,8
4,100007,0.0,Cash loans,M,N,Y,0.0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
5,"100008,0,Cash loans,M,N,Y,0,99000.0,490495.5,2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
6,100009,0.0,Cash loans,F,Y,Y,1.0,171000.0,1560726.0,41301.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,10
7,100010,0.0,Cash loans,M,Y,Y,0.0,360000.0,1530000.0,42075.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
8,100011,0.0,Cash loans,F,N,Y,0.0,112500.0,1019610.0,33826.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10
9,100012,0.0,Revolving loans,M,N,Y,0.0,135000.0,405000.0,20250.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,6


**Comment**: _Some rows have been read incorrectly and need to be handled._

In [4]:
n_rows_old = df_app.shape[0]

# incorrectly read rows
df_app_p1 = df_app[df_app['TARGET'].isnull()]

# correctly read rows
df_app_p2 = df_app[df_app['TARGET'].notnull()]

In [5]:
df_app_p2.head(10)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,random_number
1,100003,0.0,Cash loans,F,N,N,0.0,270000.000,1293502.5,35698.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,100004,0.0,Revolving loans,M,Y,Y,0.0,67500.000,135000.0,6750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,100006,0.0,Cash loans,F,N,Y,0.0,135000.000,312682.5,29686.5,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,8
4,100007,0.0,Cash loans,M,N,Y,0.0,121500.000,513000.0,21865.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
6,100009,0.0,Cash loans,F,Y,Y,1.0,171000.000,1560726.0,41301.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,10
7,100010,0.0,Cash loans,M,Y,Y,0.0,360000.000,1530000.0,42075.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
8,100011,0.0,Cash loans,F,N,Y,0.0,112500.000,1019610.0,33826.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10
9,100012,0.0,Revolving loans,M,N,Y,0.0,135000.000,405000.0,20250.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,6
10,100014,0.0,Cash loans,F,N,Y,1.0,112500.000,652500.0,21177.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4
11,100015,0.0,Cash loans,F,N,Y,0.0,38419.155,148365.0,10678.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,8


In [6]:
# reformat incorrectly read rows
df_app_p1= df_app_p1.iloc[:,0].str.split(',', expand=True)
df_app_p1 = df_app_p1[df_app_p1.columns[:-1]]
df_app_p1.columns = df_app_p2.columns

In [7]:
df_app_p1.head(10)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,random_number
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
5,100008,0,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0
18,100022,0,Revolving loans,F,N,Y,0,112500.0,157500.0,7875.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
20,100024,0,Revolving loans,M,Y,Y,0,135000.0,427500.0,21375.0,...,0,0,0,0,,,,,,
24,100030,0,Cash loans,F,N,Y,0,90000.0,225000.0,11074.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
32,100040,0,Cash loans,F,N,Y,0,135000.0,1125000.0,32895.0,...,0,0,0,0,,,,,,
33,100041,0,Cash loans,F,N,N,0,112500.0,450000.0,44509.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
36,100045,0,Cash loans,F,N,Y,0,99000.0,247275.0,17338.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
37,100047,1,Cash loans,M,N,Y,0,202500.0,1193580.0,35028.0,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,4.0
49,100060,0,Cash loans,M,Y,N,0,76500.0,454500.0,14661.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0


In [8]:
# combine all the data into one dataframe
df_app = pd.concat([df_app_p1, df_app_p2])

In [9]:
# make sure the one dataframe has the same number of rows as the original dataframe
assert(df_app.shape[0] == n_rows_old)

In [10]:
df_app.head(10)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,random_number
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
5,100008,0,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0
18,100022,0,Revolving loans,F,N,Y,0,112500.0,157500.0,7875.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
20,100024,0,Revolving loans,M,Y,Y,0,135000.0,427500.0,21375.0,...,0,0,0,0,,,,,,
24,100030,0,Cash loans,F,N,Y,0,90000.0,225000.0,11074.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
32,100040,0,Cash loans,F,N,Y,0,135000.0,1125000.0,32895.0,...,0,0,0,0,,,,,,
33,100041,0,Cash loans,F,N,N,0,112500.0,450000.0,44509.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
36,100045,0,Cash loans,F,N,Y,0,99000.0,247275.0,17338.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
37,100047,1,Cash loans,M,N,Y,0,202500.0,1193580.0,35028.0,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,4.0
49,100060,0,Cash loans,M,Y,N,0,76500.0,454500.0,14661.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
df_app.shape

(276686, 123)

### 3. Take a first look at the data - Bureau:

In [12]:
df_bureau.head(10)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.00,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.00,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
5,215354,5714467,Active,currency 1,-273,0,27460.0,NaN,0.0,0,180000.00,71017.38,108982.62,0.0,Credit card,-31,NaN
6,215354,5714468,Active,currency 1,-43,0,79.0,NaN,0.0,0,42103.80,42103.80,0.00,0.0,Consumer credit,-22,NaN
7,162297,5714469,Closed,currency 1,-1896,0,-1684.0,-1710.0,14985.0,0,76878.45,0.00,0.00,0.0,Consumer credit,-1710,NaN
8,162297,5714470,Closed,currency 1,-1146,0,-811.0,-840.0,0.0,0,103007.70,0.00,0.00,0.0,Consumer credit,-840,NaN
9,162297,5714471,Active,currency 1,-1146,0,-484.0,NaN,0.0,0,4500.00,0.00,0.00,0.0,Credit card,-690,NaN


In [13]:
df_bureau.shape

(1716428, 17)

### 4. Take a first look at the data - Column Description:

In [14]:
df_col_des.head(10)

,Unnamed: 0,Table,Row,Description,Special
0,1,application.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,"2,application.csv,TARGET,""Target variable (1 -...",NaN,NaN,NaN,NaN
2,5,application.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application.csv,CODE_GENDER,Gender of the client,NaN
4,7,application.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
5,8,application.csv,FLAG_OWN_REALTY,Flag if client owns a house or flat,NaN
6,9,application.csv,CNT_CHILDREN,Number of children the client has,NaN
7,10,application.csv,AMT_INCOME_TOTAL,Income of the client,NaN
8,11,application.csv,AMT_CREDIT,Credit amount of the loan,NaN
9,12,application.csv,AMT_ANNUITY,Loan annuity,NaN


**Comment**: _Like application data, some rows have been read incorrectly and need to be handled._

In [15]:
n_rows_old = df_col_des.shape[0]

# incorrectly read rows
df_col_des_p1 = df_col_des[df_col_des['Table'].isnull()]

# correctly read rows
df_col_des_p2 = df_col_des[df_col_des['Table'].notnull()]

df_col_des_p2.columns = ['Row_ID' if x==df_col_des.columns[0] else x for x in df_col_des_p2.columns]

In [16]:
df_col_des_p2.head(10)

,Row_ID,Table,Row,Description,Special
0,1,application.csv,SK_ID_CURR,ID of loan in our sample,NaN
2,5,application.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application.csv,CODE_GENDER,Gender of the client,NaN
4,7,application.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
5,8,application.csv,FLAG_OWN_REALTY,Flag if client owns a house or flat,NaN
6,9,application.csv,CNT_CHILDREN,Number of children the client has,NaN
7,10,application.csv,AMT_INCOME_TOTAL,Income of the client,NaN
8,11,application.csv,AMT_CREDIT,Credit amount of the loan,NaN
9,12,application.csv,AMT_ANNUITY,Loan annuity,NaN
10,13,application.csv,AMT_GOODS_PRICE,For consumer loans it is the price of the good...,NaN


In [17]:
# reformat incorrectly read rows
df_col_des_p1= df_col_des_p1.iloc[:,0].str.split(',', 3, expand=True)
df_col_des_p1['Special'] = np.nan
df_col_des_p1.columns = df_col_des_p2.columns

In [18]:
df_col_des_p1.head(10)

,Row_ID,Table,Row,Description,Special
1,2,application.csv,TARGET,"""Target variable (1 - client with payment diff...",NaN
12,15,application.csv,NAME_INCOME_TYPE,"""Clients income type (businessman, working, ma...",NaN
15,18,application.csv,NAME_HOUSING_TYPE,"""What is the housing situation of the client (...",NaN
22,25,application.csv,FLAG_MOBIL,"""Did client provide mobile phone (1=YES, 0=NO)"",",NaN
23,26,application.csv,FLAG_EMP_PHONE,"""Did client provide work phone (1=YES, 0=NO)"",",NaN
24,27,application.csv,FLAG_WORK_PHONE,"""Did client provide home phone (1=YES, 0=NO)"",",NaN
25,28,application.csv,FLAG_CONT_MOBILE,"""Was mobile phone reachable (1=YES, 0=NO)"",",NaN
26,29,application.csv,FLAG_PHONE,"""Did client provide home phone (1=YES, 0=NO)"",",NaN
27,30,application.csv,FLAG_EMAIL,"""Did client provide email (1=YES, 0=NO)"",",NaN
30,33,application.csv,REGION_RATING_CLIENT,"""Our rating of the region where client lives (...",NaN


In [19]:
# combine all the data into one dataframe
df_col_des = pd.concat([df_col_des_p1, df_col_des_p2])
df_col_des['Row_ID'] = pd.to_numeric(df_col_des['Row_ID'], errors='ignore')
df_col_des.sort_values(by='Row_ID', ascending=True, inplace=True)

In [20]:
# make sure the one dataframe has the same number of rows as the original dataframe
assert(df_col_des.shape[0] == n_rows_old)

In [21]:
df_col_des.head(10)

,Row_ID,Table,Row,Description,Special
0,1,application.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application.csv,TARGET,"""Target variable (1 - client with payment diff...",NaN
2,5,application.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application.csv,CODE_GENDER,Gender of the client,NaN
4,7,application.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
5,8,application.csv,FLAG_OWN_REALTY,Flag if client owns a house or flat,NaN
6,9,application.csv,CNT_CHILDREN,Number of children the client has,NaN
7,10,application.csv,AMT_INCOME_TOTAL,Income of the client,NaN
8,11,application.csv,AMT_CREDIT,Credit amount of the loan,NaN
9,12,application.csv,AMT_ANNUITY,Loan annuity,NaN


In [22]:
df_col_des.to_csv("columns_description(cleaned).csv", index = False)

### 5. Summarize the dataframe

In [23]:
df_app.describe([.1,.2,.3,.6,.7,.8,.9,.95,.98,.99,.999])

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,random_number
count,276686,276686.0,276686,276686,276686,276686,276686.0,276686.0,276686.0,276678.0,...,276686.0,276686.0,276686.0,247998.0,247998.0,247998.0,247998.0,247998.0,247998.0,276686
unique,276686,4.0,2,3,2,2,23.0,2833.0,8765.0,22502.0,...,4.0,4.0,4.0,7.0,14.0,17.0,31.0,29.0,40.0,30
top,215264,0.0,Cash loans,F,N,Y,0.0,135000.0,450000.0,9000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
freq,1,192643.0,250263,182057,182593,191871,147155.0,24386.0,6601.0,4349.0,...,209914.0,209944.0,209965.0,180244.0,180355.0,175533.0,151871.0,146779.0,48345.0,23599


**Comment**: _No statistics printed out, this means all data have been stored previously as string, preprocessing neededd._

In [24]:
# One example
df_app['AMT_INCOME_TOTAL'][0]

'202500.0'

In [25]:
# convert string to numeric if the data is numeric, otherwise keep original string version
for col in df_app.columns:
    df_app[col] = pd.to_numeric(df_app[col], errors='ignore')

In [26]:
df_app.describe([.1,.2,.3,.6,.7,.8,.9,.95,.98,.99,.999])

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,random_number
count,276686.000000,276686.000000,276686.000000,2.766860e+05,2.766860e+05,276675.000000,2.764270e+05,276686.000000,276686.000000,276686.000000,...,276686.000000,276686.000000,276686.000000,247998.000000,239635.000000,239386.000000,239386.000000,239386.000000,239386.000000,268074.000000
mean,278228.098939,0.080824,0.416013,1.688559e+05,5.987201e+05,27100.156769,5.381233e+05,-15461.279088,61415.656513,-2840.702092,...,0.002356,0.000549,0.000419,0.004762,0.006685,0.027771,0.206077,0.267509,1.508969,5.096723
std,102800.236957,0.272566,0.719639,2.471935e+05,4.024394e+05,14490.114074,3.693735e+05,5251.822942,139918.505204,27832.315504,...,0.048486,0.023432,0.020471,0.072662,0.103747,0.185793,0.793481,0.703983,1.873693,2.982774
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,-25229.000000,-25152.000000,-23738.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10%,135688.500000,0.000000,0.000000,8.100000e+04,1.800000e+05,11070.000000,1.800000e+05,-22125.000000,-5934.000000,-9878.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
20%,171375.000000,0.000000,0.000000,9.900000e+04,2.547000e+05,14679.000000,2.250000e+05,-20374.000000,-3605.000000,-8149.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
30%,207168.000000,0.000000,0.000000,1.125000e+05,3.060000e+05,18157.500000,2.700000e+05,-18734.000000,-2566.000000,-6657.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
50%,278256.500000,0.000000,0.000000,1.485000e+05,5.130000e+05,24898.500000,4.500000e+05,-15511.000000,-1296.000000,-4378.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000
60%,313896.000000,0.000000,0.000000,1.620000e+05,6.041520e+05,28062.000000,5.220000e+05,-14147.000000,-875.000000,-3380.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.000000
70%,349493.500000,0.000000,0.000000,1.800000e+05,7.551900e+05,32017.500000,6.750000e+05,-12764.000000,-494.000000,-2394.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,7.000000


In [27]:
df_bureau.describe([.1,.2,.3,.6,.7,.8,.9,.95,.98,.99,.999])

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
count,1.716428e+06,1.716428e+06,1.716428e+06,1.716428e+06,1.610875e+06,1.082775e+06,5.919400e+05,1.716428e+06,1.716415e+06,1.458759e+06,1.124648e+06,1.716428e+06,1.716428e+06,4.896370e+05
mean,2.782149e+05,5.924434e+06,-1.142108e+03,8.181666e-01,5.105174e+02,-1.017437e+03,3.825418e+03,6.410406e-03,3.549946e+05,1.370851e+05,6.229515e+03,3.791276e+01,-5.937483e+02,1.571276e+04
std,1.029386e+05,5.322657e+05,7.951649e+02,3.654443e+01,4.994220e+03,7.140106e+02,2.060316e+05,9.622391e-02,1.149811e+06,6.774011e+05,4.503203e+04,5.937650e+03,7.207473e+02,3.258269e+05
min,1.000010e+05,5.000000e+06,-2.922000e+03,0.000000e+00,-4.206000e+04,-4.202300e+04,0.000000e+00,0.000000e+00,0.000000e+00,-4.705600e+06,-5.864061e+05,0.000000e+00,-4.194700e+04,0.000000e+00
10%,1.356020e+05,5.184875e+06,-2.443000e+03,0.000000e+00,-1.922000e+03,-2.159000e+03,0.000000e+00,0.000000e+00,2.250000e+04,0.000000e+00,0.000000e+00,0.000000e+00,-1.561000e+03,0.000000e+00
20%,1.712200e+05,5.370915e+06,-1.879000e+03,0.000000e+00,-1.357000e+03,-1.677000e+03,0.000000e+00,0.000000e+00,4.276260e+04,0.000000e+00,0.000000e+00,0.000000e+00,-1.039000e+03,0.000000e+00
30%,2.067270e+05,5.556600e+06,-1.501000e+03,0.000000e+00,-9.530000e+02,-1.325000e+03,0.000000e+00,0.000000e+00,6.511950e+04,0.000000e+00,0.000000e+00,0.000000e+00,-7.970000e+02,0.000000e+00
50%,2.780550e+05,5.926304e+06,-9.870000e+02,0.000000e+00,-3.300000e+02,-8.970000e+02,0.000000e+00,0.000000e+00,1.255185e+05,0.000000e+00,0.000000e+00,0.000000e+00,-3.950000e+02,0.000000e+00
60%,3.140000e+05,6.109587e+06,-7.640000e+02,0.000000e+00,-5.900000e+01,-6.990000e+02,0.000000e+00,0.000000e+00,1.715850e+05,0.000000e+00,0.000000e+00,0.000000e+00,-1.830000e+02,5.125500e+03
70%,3.495490e+05,6.293764e+06,-5.670000e+02,0.000000e+00,2.480000e+02,-5.110000e+02,0.000000e+00,0.000000e+00,2.392650e+05,0.000000e+00,0.000000e+00,0.000000e+00,-5.100000e+01,1.045403e+04


**Comment**: _bureau data seems normal._

## 2. Preprocessing

### 1. Focus on bureau data first and turn it into features for application

In [28]:
def most_common(series):
    return series.value_counts().index[0]

In [29]:
df_bureau_grouped = df_bureau.groupby(['SK_ID_CURR']).agg({'CREDIT_ACTIVE': most_common, 
                                                           'CREDIT_CURRENCY': most_common, 
                                                           'DAYS_CREDIT': 'median', 
                                                           'CREDIT_DAY_OVERDUE': 'median', 
                                                           'DAYS_CREDIT_ENDDATE': 'median', 
                                                           'DAYS_ENDDATE_FACT': 'median', 
                                                           'AMT_CREDIT_MAX_OVERDUE': 'median', 
                                                           'CNT_CREDIT_PROLONG': 'median', 
                                                           'AMT_CREDIT_SUM': 'median', 
                                                           'AMT_CREDIT_SUM_DEBT': 'median', 
                                                           'AMT_CREDIT_SUM_LIMIT': 'median', 
                                                           'AMT_CREDIT_SUM_OVERDUE': 'median',
                                                           'CREDIT_TYPE': most_common, 
                                                           'DAYS_CREDIT_UPDATE': 'median', 
                                                           'AMT_ANNUITY': 'median'}).reset_index()

In [30]:
df_bureau_grouped.head()

,SK_ID_CURR,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,100001,Closed,currency 1,-857.0,0.0,-179.0,-715.0,NaN,0.0,168345.00,0.0,0.0,0.0,Consumer credit,-155.0,0.0
1,100002,Closed,currency 1,-1042.5,0.0,-424.5,-939.0,40.5,0.0,54130.50,0.0,0.0,0.0,Consumer credit,-402.5,0.0
2,100003,Closed,currency 1,-1205.5,0.0,-480.0,-621.0,0.0,0.0,92576.25,0.0,0.0,0.0,Consumer credit,-545.0,NaN
3,100004,Closed,currency 1,-867.0,0.0,-488.5,-532.5,0.0,0.0,94518.90,0.0,0.0,0.0,Consumer credit,-532.0,NaN
4,100005,Active,currency 1,-137.0,0.0,122.0,-123.0,0.0,0.0,58500.00,25321.5,0.0,0.0,Consumer credit,-31.0,0.0


### 2. Merge application and bureau data

In [31]:
df_merged = df_app.merge(df_bureau_grouped, on='SK_ID_CURR', how='left', suffixes=('', '_BUREAU'))

In [32]:
df_merged.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY_BUREAU
0,100002,1.0,Cash loans,M,N,Y,0.0,202500.0,406597.5,24700.5,...,-939.0,40.5,0.0,54130.500,0.0,0.0,0.0,Consumer credit,-402.5,0.0
1,100008,0.0,Cash loans,M,N,Y,0.0,99000.0,490495.5,27517.5,...,-909.0,0.0,0.0,105705.000,0.0,0.0,0.0,Consumer credit,-790.0,NaN
2,100022,0.0,Revolving loans,F,N,Y,0.0,112500.0,157500.0,7875.0,...,NaN,0.0,0.0,528750.000,205276.5,0.0,0.0,Consumer credit,-28.0,NaN
3,100024,0.0,Revolving loans,M,Y,Y,0.0,135000.0,427500.0,21375.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100030,0.0,Cash loans,F,N,Y,0.0,90000.0,225000.0,11074.5,...,-598.0,0.0,0.0,33487.785,0.0,0.0,0.0,Consumer credit,-244.0,NaN


In [33]:
df_merged.to_csv("app_merged.csv", index = False)

### 3. Correlation and remove columns with high correlation

In [34]:
corr = df_merged.corr()
corr.style.background_gradient(cmap='coolwarm')

In [35]:
def corr_handler(df, threshold1, threshold2):
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = df.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold1) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in df.columns:
                    del df[colname] # deleting the column from the dataset
            if (corr_matrix.iloc[i, j] <= threshold2) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in df.columns:
                    del df[colname] # deleting the column from the dataset

    return df

df_merged =  corr_handler(df_merged, 0.85, -0.85)

In [36]:
df_merged.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY_BUREAU
0,100002,1.0,Cash loans,M,N,Y,0.0,202500.0,406597.5,24700.5,...,-939.0,40.5,0.0,54130.500,0.0,0.0,0.0,Consumer credit,-402.5,0.0
1,100008,0.0,Cash loans,M,N,Y,0.0,99000.0,490495.5,27517.5,...,-909.0,0.0,0.0,105705.000,0.0,0.0,0.0,Consumer credit,-790.0,NaN
2,100022,0.0,Revolving loans,F,N,Y,0.0,112500.0,157500.0,7875.0,...,NaN,0.0,0.0,528750.000,205276.5,0.0,0.0,Consumer credit,-28.0,NaN
3,100024,0.0,Revolving loans,M,Y,Y,0.0,135000.0,427500.0,21375.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100030,0.0,Cash loans,F,N,Y,0.0,90000.0,225000.0,11074.5,...,-598.0,0.0,0.0,33487.785,0.0,0.0,0.0,Consumer credit,-244.0,NaN


### 4. Check if there are missing values in the columns:

In [37]:
# Check if there are missing values in the columns:
def missing_value_checker(df):
    for col in df.columns:
        tmp_flag = 'Numerical'
        if df[col].dtype == object:
            tmp_flag = 'Categorical'
        logging.info('{}: {} ({})'.format(col, str(df[col].isnull().sum()/float(df.shape[0])), tmp_flag))

In [38]:
missing_value_checker(df_merged)

INFO:root:SK_ID_CURR: 0.0 (Numerical)
INFO:root:TARGET: 0.0 (Numerical)
INFO:root:NAME_CONTRACT_TYPE: 0.0 (Categorical)
INFO:root:CODE_GENDER: 0.0 (Categorical)
INFO:root:FLAG_OWN_CAR: 0.0 (Categorical)
INFO:root:FLAG_OWN_REALTY: 0.0 (Categorical)
INFO:root:CNT_CHILDREN: 0.0 (Numerical)
INFO:root:AMT_INCOME_TOTAL: 0.0 (Numerical)
INFO:root:AMT_CREDIT: 0.0 (Numerical)
INFO:root:AMT_ANNUITY: 3.975625799642916e-05 (Numerical)
INFO:root:NAME_TYPE_SUITE: 0.0032202568977107624 (Categorical)
INFO:root:NAME_INCOME_TYPE: 0.0 (Categorical)
INFO:root:NAME_EDUCATION_TYPE: 0.0 (Categorical)
INFO:root:NAME_FAMILY_STATUS: 0.0 (Categorical)
INFO:root:NAME_HOUSING_TYPE: 0.0 (Categorical)
INFO:root:REGION_POPULATION_RELATIVE: 0.0 (Categorical)
INFO:root:DAYS_BIRTH: 0.0 (Numerical)
INFO:root:DAYS_EMPLOYED: 0.0 (Numerical)
INFO:root:DAYS_REGISTRATION: 0.0 (Numerical)
INFO:root:DAYS_ID_PUBLISH: 0.0 (Numerical)
INFO:root:OWN_CAR_AGE: 0.6386915131231793 (Numerical)
INFO:root:FLAG_MOBIL: 0.021255141206999992 

**Comment**: _Missing Data in the dataframe found, attention needed._

### 5. See how many unique values there are for each column:

In [39]:
# See how many unique values there are for each columns, and if there is only one unique value, we drop the column
def unique_value_printer(df):
    cols_to_drop = []
    for col in df.columns:
        tmp_num_unique = len(df[col].unique())
        tmp_flag = 'Numerical'
        if df[col].dtype == object:
            tmp_flag = 'Categorical'
        logging.info('{}: {} ({})'.format(col, str(tmp_num_unique), tmp_flag))
        if (tmp_num_unique==1):
            cols_to_drop.append(col)
    df.drop(cols_to_drop, 1, inplace=True)

In [40]:
unique_value_printer(df_merged)

INFO:root:SK_ID_CURR: 276686 (Numerical)
INFO:root:TARGET: 2 (Numerical)
INFO:root:NAME_CONTRACT_TYPE: 2 (Categorical)
INFO:root:CODE_GENDER: 3 (Categorical)
INFO:root:FLAG_OWN_CAR: 2 (Categorical)
INFO:root:FLAG_OWN_REALTY: 2 (Categorical)
INFO:root:CNT_CHILDREN: 14 (Numerical)
INFO:root:AMT_INCOME_TOTAL: 2348 (Numerical)
INFO:root:AMT_CREDIT: 5439 (Numerical)
INFO:root:AMT_ANNUITY: 13418 (Numerical)
INFO:root:NAME_TYPE_SUITE: 9 (Categorical)
INFO:root:NAME_INCOME_TYPE: 9 (Categorical)
INFO:root:NAME_EDUCATION_TYPE: 10 (Categorical)
INFO:root:NAME_FAMILY_STATUS: 11 (Categorical)
INFO:root:NAME_HOUSING_TYPE: 11 (Categorical)
INFO:root:REGION_POPULATION_RELATIVE: 166 (Categorical)
INFO:root:DAYS_BIRTH: 17513 (Numerical)
INFO:root:DAYS_EMPLOYED: 17766 (Numerical)
INFO:root:DAYS_REGISTRATION: 15534 (Numerical)
INFO:root:DAYS_ID_PUBLISH: 8686 (Numerical)
INFO:root:OWN_CAR_AGE: 4538 (Numerical)
INFO:root:FLAG_MOBIL: 46 (Numerical)
INFO:root:FLAG_WORK_PHONE: 2 (Numerical)
INFO:root:FLAG_CONT

### 6. Encoding of the categorical features

**Comment**: _Some categorical features have hundreds, or even thousands (e.g. TOTALAREA_MODE, WALLSMATERIAL_MODE), this means one hot encoding is not suitable here. Due to the presence of nominal categorical features, label encoding will also be a bad idea._

We use frequency encoding here to ensure scalability of the encoding method and help avoid the explosion of dimensionality.

In [41]:
for col in df_merged.columns:
    if df_merged[col].dtype == object:
        df_tmp_group = df_merged.groupby(col).size()/len(df_merged)
        df_merged.loc[:, col+'_ENCODED'] = df_merged[col].map(df_tmp_group)

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


### 7. Normalize the data

In [42]:
def normalize(df, cols_to_ignore):
    for col in df.columns:
        if df[col].dtype == object or col in cols_to_ignore:
            pass
        else:
            max_value = df[col].max()
            min_value = df[col].min()
            if max_value > min_value:
                df[col] = (df[col] - min_value) / (max_value - min_value)
    return df
df_merged = normalize(df_merged, ['SK_ID_CURR', 'TARGET'])

In [43]:
df_merged.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FONDKAPREMONT_MODE_ENCODED,HOUSETYPE_MODE_ENCODED,TOTALAREA_MODE_ENCODED,WALLSMATERIAL_MODE_ENCODED,EMERGENCYSTATE_MODE_ENCODED,OBS_30_CNT_SOCIAL_CIRCLE_ENCODED,DEF_30_CNT_SOCIAL_CIRCLE_ENCODED,CREDIT_ACTIVE_ENCODED,CREDIT_CURRENCY_ENCODED,CREDIT_TYPE_ENCODED
0,100002,1.0,Cash loans,M,N,Y,0.0,0.001512,0.090287,0.090032,...,1.000000,1.000000,0.011948,0.982035,0.682701,0.525811,0.033541,1.000000,1.0,1.0
1,100008,0.0,Cash loans,M,N,Y,0.0,0.000627,0.111235,0.101018,...,0.436712,0.062619,1.000000,0.096320,0.068980,0.048604,0.186867,1.000000,1.0,1.0
2,100022,0.0,Revolving loans,F,N,Y,0.0,0.000742,0.028090,0.024412,...,1.000000,1.000000,0.005887,0.982035,0.682701,0.525811,0.003448,0.387717,1.0,1.0
3,100024,0.0,Revolving loans,M,Y,Y,0.0,0.000935,0.095506,0.077062,...,1.000000,1.000000,0.014892,0.982035,0.682701,0.525811,0.186867,NaN,NaN,NaN
4,100030,0.0,Cash loans,F,N,Y,0.0,0.000550,0.044944,0.036890,...,1.000000,1.000000,0.017489,0.982035,0.682701,0.525811,0.186867,1.000000,1.0,1.0


### 8. Fill the missing values with Multiple Imputation by Chained Equations(MICE)

In [45]:
features_drop = ['SK_ID_CURR', 'TARGET', 'TARGET_BOOL']
features_train = []
for col in df_merged.columns:
    if col in features_drop or df_merged[col].dtype == object:
        pass
    else:
        features_train.append(col)
        
df_X = df_merged[features_train]

In [47]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imputer = IterativeImputer(imputation_order='ascending',max_iter=10,random_state=42,n_nearest_features=None)
df_filled = imputer.fit_transform(df_X) # np.array format

# generate new imputed dataframe
df_X = pd.DataFrame(data=df_filled, index=df_X.index, columns=df_X.columns)

/home/yao/anaconda3/lib/python3.7/site-packages/sklearn/impute/_iterative.py:638: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


In [48]:
df_X.to_csv("app_final.csv", index = False)

In [52]:
df_X['TARGET'] = df_merged['TARGET']
df_y = df_merged['TARGET']

df_X.to_csv("app_final(w_target).csv", index = False) # for FFTrees R code

## 3. Comparative Model Analysis

In [53]:
import logging

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("Start 3. Comparative Model Analysis")

df_merged = pd.read_csv("app_merged.csv", sep = ",")
df_X = pd.read_csv("app_final.csv", sep = ",")
df_y = df_merged['TARGET']

INFO:root:Start 3. Comparative Model Analysis
/home/yao/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (16,29,33,41,88,91,92) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 1. FFT

**Comment**: _FFT has been run with the original FFTrees library in R, the script: r_fftrees.r,
the results can be seen in the presentation._


### 2. Gradient Boosting

In [5]:
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import cross_val_score
import warnings

def xgboostcv(max_depth,
              learning_rate,
              n_estimators,
              gamma,
              min_child_weight,
              max_delta_step,
              subsample,
              colsample_bytree,
              reg_alpha,
              reg_lambda,
              silent=True,
              nthread=-1,
              random_state=1):
    return cross_val_score(xgb.XGBClassifier(max_depth=int(max_depth),
                                             learning_rate=learning_rate,
                                             n_estimators=int(n_estimators),
                                             silent=silent,
                                             nthread=nthread,
                                             gamma=gamma,
                                             min_child_weight=min_child_weight,
                                             max_delta_step=max_delta_step,
                                             subsample=subsample,
                                             colsample_bytree=colsample_bytree,
                                             reg_alpha=reg_alpha,
                                             reg_lambda = reg_lambda),
                           df_X,
                           df_y,
                           cv=10,
                           scoring="balanced_accuracy",
                           n_jobs=-1).mean()

xgboostBO = BayesianOptimization(xgboostcv,
                                 {'max_depth': (2, 5),
                                  'learning_rate': (0.01, 0.3),
                                  'n_estimators': (1000, 2500),
                                  'gamma': (1., 0.01),
                                  'min_child_weight': (1, 10),
                                  'max_delta_step': (0, 0.1),
                                  'subsample': (0.5, 0.8),
                                  'colsample_bytree' :(0.1, 0.99),
                                  'reg_alpha':(0.1, 0.5),
                                  'reg_lambda':(0.1, 0.9)
                                  })

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    xgboostBO.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)

|   iter    |  target   | colsam... |   gamma   | learni... | max_de... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.5122   |  0.6706   |  0.7123   |  0.1106   |  0.05343  |  2.658    |  3.329    |  1.864e+0 |  0.1689   |  0.7458   |  0.6665   |
|  2        |  0.5      |  0.1097   |  0.06219  |  0.01192  |  0.09386  |  4.016    |  5.055    |  2.392e+0 |  0.383    |  0.8555   |  0.7437   |
|  3        |  0.512    |  0.5241   |  0.01     |  0.2822   |  0.03741  |  4.392    |  4.106    |  1e+03    |  0.4843   |  0.8353   |  0.6309   |
|  4        |  0.5125   |  0.7417   |  0.01     |  0.1308   |  0.07121  |  2.074    |  1.097    |  1.439e+0 |  0.2653   |  0.7681   |  0.7233   |
|  5        |  0.5      |  0.6578   |  0.01     |  0.1155   |  0.008655 |  4.594    |  9.954    |  1.446e+0 |  0.3254   |  0

In [6]:
logging.info('-'*100)
logging.info('Final Results')
logging.info('Maximum XGBOOST value: %f' % xgboostBO.max['target'])
logger.info("Loging dict ---> {0}".format(xgboostBO.max['params']))

INFO:root:----------------------------------------------------------------------------------------------------
INFO:root:Final Results
INFO:root:Maximum XGBOOST value: 0.512540
INFO:root:Loging dict ---> {'colsample_bytree': 0.7416874301754368, 'gamma': 0.01, 'learning_rate': 0.13084370343179114, 'max_delta_step': 0.07120556905422434, 'max_depth': 2.0742752578632473, 'min_child_weight': 1.097028205883142, 'n_estimators': 1439.0264972375603, 'reg_alpha': 0.2653395278375019, 'reg_lambda': 0.7681197379733655, 'subsample': 0.7233116592694329}


In [7]:
max_params = xgboostBO.max['params']
max_params['max_depth'] = int(max_params['max_depth'])

xgb_final_model = xgb.XGBClassifier(max_depth=max_params['max_depth'],
                                             learning_rate=max_params['learning_rate'],
                                             n_estimators=int(max_params['n_estimators']),
                                             silent=True,
                                             nthread=-1,
                                             gamma=max_params['gamma'],
                                             min_child_weight=max_params['min_child_weight'],
                                             max_delta_step=max_params['max_delta_step'],
                                             subsample=max_params['subsample'],
                                             colsample_bytree=max_params['colsample_bytree'],
                                             reg_alpha=max_params['reg_alpha'],
                                             reg_lambda = max_params['reg_lambda'])

In [8]:
cv_xgboost = cross_val_score(xgb_final_model, 
                df_X, 
                df_y, 
                cv=10,
                scoring="balanced_accuracy", 
                n_jobs=-1)

INFO:root:BACC for the fold no. 1 on the test set: 0.5090527452170992
INFO:root:BACC for the fold no. 2 on the test set: 0.5088688109905699
INFO:root:BACC for the fold no. 3 on the test set: 0.5120142092196759
INFO:root:BACC for the fold no. 4 on the test set: 0.5171449955647546
INFO:root:BACC for the fold no. 5 on the test set: 0.5119697822034894
INFO:root:BACC for the fold no. 6 on the test set: 0.5132979031826448
INFO:root:BACC for the fold no. 7 on the test set: 0.5147275468186205
INFO:root:BACC for the fold no. 8 on the test set: 0.5124593744358319
INFO:root:BACC for the fold no. 9 on the test set: 0.514374920914023
INFO:root:BACC for the fold no. 10 on the test set: 0.5146536696051214


### 3. Random Forest

In [ ]:
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import cross_val_score
import warnings
from sklearn.ensemble import RandomForestClassifier

def rfccv(n_estimators, min_samples_split, max_features):
    return cross_val_score(RandomForestClassifier(n_estimators=int(n_estimators),
                                                  min_samples_split=int(min_samples_split),
                                                  max_features=min(max_features, 0.999)), 
                           df_X, 
                           df_y, 
                           cv=10,
                           scoring="balanced_accuracy", 
                           n_jobs=-1).mean()


rfcBO = BayesianOptimization(rfccv, {'n_estimators': (10, 25),
                                     'min_samples_split': (2, 20),
                                     'max_features': (0.1, 0.999)})

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    rfcBO.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)
    
logging.info('-'*100)
logging.info('Final Results')
logging.info('Maximum RF value: %f' % rfcBO.max['target'])
logger.info("Loging dict ---> {0}".format(rfcBO.max['params']))

|   iter    |  target   | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------
|  1        |  0.5059   |  0.4447   |  17.57    |  23.61    |
|  2        |  0.5047   |  0.2018   |  3.488    |  23.75    |
|  3        |  0.5087   |  0.3052   |  19.66    |  10.06    |
|  4        |  0.5119   |  0.8397   |  19.7     |  10.1     |


In [ ]:
max_params = rfcBO.max['params']
max_params['max_depth'] = int(max_params['max_depth'])

rf_final_model = RandomForestClassifier(n_estimators=int(max_params['n_estimators']),
                                             min_samples_split=int(max_params['min_samples_split']),
                                             max_features=min(max_params['max_features'], 0.999))

In [ ]:
cv_rf = cross_val_score(rf_final_model, 
                df_X, 
                df_y, 
                cv=10,
                scoring="balanced_accuracy", 
                n_jobs=-1)